# Data Science I Project

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


## Utils

In [ ]:
def create_frequency_table_with_credit_risk(index: str, df: pd.DataFrame):
    cross_table_credit_hist = pd.crosstab(df[index], df["CreditRiskClass"], margins=True)
    cross_table_credit_hist[f"P(Bad|{index})"] = cross_table_credit_hist["Bad"]/cross_table_credit_hist["All"]
    cross_table_credit_hist[f"P(Good|{index})"] = 1 - cross_table_credit_hist[f"P(Bad|{index})"]
    return cross_table_credit_hist

def detect_rare_categories(dataset_df: pd.DataFrame, threshold: int):
    categorical_cols = dataset_df.select_dtypes("object").columns
    
    for col_name in categorical_cols:
        rel_freq_s = dataset_df[col_name].value_counts(normalize=True)*100
        
        below_threshold = rel_freq_s[rel_freq_s < threshold].index.to_list()
        
        if (len(below_threshold)): 
            print(f"\n Categories in Feature {col_name} below threshold freq: {below_threshold}")
            print("Relative Frequency: ")
            print(rel_freq_s)

def merge_categories(dataset_df: pd.DataFrame, feature: str, categories_to_merge: list, merged_category_name: str):
    dataset_df[feature] = dataset_df[feature].apply(lambda value: merged_category_name if value in categories_to_merge else value)
    return dataset_df

def detect_rare_ordinals(dataset_df: pd.DataFrame, threshold: int):
    numerical_cols = dataset_df.select_dtypes("int64")
    
    col_unique_counts_s = numerical_cols.nunique()
    
    ordinal_cols = col_unique_counts_s[col_unique_counts_s < 8].index
    
    for col_name in ordinal_cols:
        rel_freq_s = dataset_df[col_name].value_counts(normalize=True)*100
        
        below_threshold = rel_freq_s[rel_freq_s < threshold].index.to_list()
        
        if (len(below_threshold)): 
            print(f"\n Categories in Feature {col_name} below threshold freq: {below_threshold}")
            print("Relative Frequency: ")
            print(rel_freq_s)

## Loading Dataset

In [ ]:
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144)

In [ ]:
statlog_german_credit_data.data.original

In [ ]:
metadata = statlog_german_credit_data.metadata

print("Abstract: ", metadata["abstract"])
print("Area: ", metadata["area"])
print("Number of Instances: ", metadata["num_instances"])
print("Has Missing Values: ", metadata["has_missing_values"])
print("Number of Features: ", metadata["num_features"])
print("Feature Types: ", metadata["feature_types"])
print("Demographics: ", metadata["demographics"])

In [ ]:
statlog_german_credit_data.variables

## Data Pre-processing (with some EDA)

In this part, the categorical features will be transformed into easier understandable data, so that the EDA can be performed.
Besides, categories in some features will be joined to avoid overfitting.

In [ ]:
dataset_df = statlog_german_credit_data.data.original

COLUMN_RENAME_DICT = {
    'Attribute1': 'CheckingAccountStatus',
    'Attribute2': 'DurationInMonths',
    'Attribute3': 'CreditHistory',
    'Attribute4': 'Purpose',
    'Attribute5': 'CreditAmount',
    'Attribute6': 'SavingsAccountBonds',
    'Attribute7': 'EmploymentSince',
    'Attribute8': 'InstallmentRate',
    'Attribute9': 'PersonalStatusSex',
    'Attribute10': 'OtherDebtorsGuarantors',
    'Attribute11': 'ResidenceSince',
    'Attribute12': 'Property',
    'Attribute13': 'Age',
    'Attribute14': 'OtherInstallmentPlans',
    'Attribute15': 'Housing',
    'Attribute16': 'ExistingCreditsCount',
    'Attribute17': 'Job',
    'Attribute18': 'PeopleLiableMaintenance',
    'Attribute19': 'Telephone',
    'Attribute20': 'ForeignWorker',
    'class': 'CreditRiskClass'
}

dataset_df = dataset_df.rename(columns = COLUMN_RENAME_DICT)


Below, the mappings for the categorical data are defined

In [ ]:
# Consolidated mapping dictionary for all categorical columns
categorical_mappings = {
    'CheckingAccountStatus': {
        'A11': 'Less than 0 DM',
        'A12': '0 to 200 DM',
        'A13': '200 DM or more',
        'A14': 'No checking account'
    },
    'CreditHistory': {
        'A30': 'No credits taken', # individual has never taken out any credits
        'A31': 'All credits at this bank paid back duly', # individual has taken out credits from this particular bank and has paid them all back on time
        'A32': 'Existing credits paid back duly till now', # individual currently has existing credits and has been paying them back on time up to the present moment.
        'A33': 'Delay in paying off in the past', # individual has experienced delays in making credit payments in the past
        'A34': 'Critical account / other credits existing'
    },
    'Purpose': {
        'A40': 'Car (new)',
        'A41': 'Car (used)',
        'A42': 'Furniture/equipment',
        'A43': 'Radio/television',
        'A44': 'Domestic appliances',
        'A45': 'Repairs',
        'A46': 'Education',
        'A47': 'Vacation',
        'A48': 'Retraining',
        'A49': 'Business',
        'A410': 'Others'
    },
    'SavingsAccountBonds': {
        'A61': 'Less than 100 DM',
        'A62': '100 to 500 DM',
        'A63': '500 to 1000 DM',
        'A64': '1000 DM or more',
        'A65': 'Unknown / no savings account'
    },
    'EmploymentSince': {
        'A71': 'Unemployed',
        'A72': 'Less than 1 year',
        'A73': '1 to 4 years',
        'A74': '4 to 7 years',
        'A75': '7 years or more'
    },
    'PersonalStatusSex': {
        'A91': 'Male: divorced',
        'A92': 'Female: divorced/married',
        'A93': 'Male: single',
        'A94': 'Male: married/widowed',
        'A95': 'Female: single'
    },
    'OtherDebtorsGuarantors': {
        'A101': 'None',
        'A102': 'Co-applicant',
        'A103': 'Guarantor'
    },
    'Property': {
        'A121': 'Real estate',
        'A122': 'Building society savings agreement / life insurance',
        'A123': 'Car or other, not in attribute 6',
        'A124': 'Unknown / no property'
    },
    'OtherInstallmentPlans': {
        'A141': 'Bank',
        'A142': 'Stores',
        'A143': 'None'
    },
    'Housing': {
        'A151': 'Rent',
        'A152': 'Own',
        'A153': 'For free'
    },
    'Job': {
        'A171': 'Unemployed / unskilled - non-resident',
        'A172': 'Unskilled - resident',
        'A173': 'Skilled employee / official',
        'A174': 'Management / self-employed / \n highly qualified employee / officer'
    },
    'Telephone': {
        'A191': 'None',
        'A192': 'Yes, registered under the customer\'s name'
    },
    'ForeignWorker': {
        'A201': 'Yes',
        'A202': 'No'
    }
}

In [ ]:
# replacing codes with descriptive labels using the consolidated mappings
for column, mapping in categorical_mappings.items():
    dataset_df[column] = dataset_df[column].replace(mapping)

# Separating Personal Status from Gender in original feature "PersonalStatusSex" 
dataset_df["Gender"] = dataset_df["PersonalStatusSex"].str.extract("([A-Za-z]+)")
dataset_df["PersonalStatus"] = dataset_df["PersonalStatusSex"].str.extract("(?<=:\s)(.*)$")
dataset_df = dataset_df.drop("PersonalStatusSex", axis=1)

# Replace values of the target variable, as defined in the instructions of the dataset 
dataset_df["CreditRiskClass"] = dataset_df["CreditRiskClass"].replace(1,0) # Good payer, low risk
dataset_df["CreditRiskClass"] = dataset_df["CreditRiskClass"].replace(2,1) # Bad payer. high risk

credit_risk_class_map = ["Good", "Bad"]

dataset_df["CreditRiskClass"] = dataset_df["CreditRiskClass"].apply(lambda risk_class: credit_risk_class_map[risk_class])

print("New columns: ", dataset_df.columns)

In [ ]:
detect_rare_categories(dataset_df, 5)

In [ ]:
# Rare categories are merged to reduce complexity and avoid overfitting

dataset_df = merge_categories(dataset_df, feature="CreditHistory",
                 categories_to_merge=["All credits at this bank paid back duly", "No credits taken"],
                 merged_category_name="No credits taken or all credits paid back duly")

dataset_df = merge_categories(dataset_df, feature="Purpose",
                 categories_to_merge=["Retraining", "Domestic appliances", "Repairs", "Education"],
                 merged_category_name="Others")

dataset_df = merge_categories(dataset_df, feature="SavingsAccountBonds",
                              categories_to_merge=["500 to 1000 DM", "1000 DM or more"],
                              merged_category_name="500 DM or more")

dataset_df = merge_categories(dataset_df, feature="OtherDebtorsGuarantors",
                 categories_to_merge=["Guarantor", "Co-applicant"],
                 merged_category_name="Yes")
dataset_df["OtherDebtorsGuarantors"] = dataset_df["OtherDebtorsGuarantors"].replace("None", "No")

dataset_df = merge_categories(dataset_df, feature="OtherInstallmentPlans",
                 categories_to_merge=["Bank", "Stores"],
                 merged_category_name="Yes")
dataset_df["OtherInstallmentPlans"] = dataset_df["OtherInstallmentPlans"].replace("None", "No")

dataset_df = merge_categories(dataset_df, feature="Job",
                 categories_to_merge=["Unskilled - resident", "Unemployed / unskilled - non-resident"],
                 merged_category_name="Unemployed/Unskilled")
dataset_df["OtherInstallmentPlans"] = dataset_df["OtherInstallmentPlans"].replace("None", "No")


Although only 37 samples out of 1000 have the ForeignWorker = 1, this feature could still be useful for the credit risk prediction. However, This is a sign that this feature could potentially lead to overfitting. Future steps will evaluate if this feature is necessary or not.

In [ ]:
detect_rare_ordinals(dataset_df, 5)

## EDA

In [ ]:
print(dataset_df.describe())

In [ ]:
print(dataset_df.info())
number_samples = len(dataset_df)

In [ ]:
sns.countplot(x="CreditRiskClass", data=dataset_df, hue="CreditRiskClass")
dataset_df["CreditRiskClass"].value_counts()/number_samples

The dataset is highly unbalanced between the classes "Good payers" and "Bad payers"

### Analysis of Numerical Features

In [ ]:
sns.pairplot(dataset_df, hue="CreditRiskClass")

#### 1. Univariate distributions
Observing the diagonal, no features can clearly separate the two classes, what makes the classification task harder. The univariate distributions for both classes are general very similar. Apparently, the features "CreditAmount" and "DurationInMonths" are the ones how gives a higher difference in the distribution format.

#### 2. DurationInMonths and CreditAmount
In the features 'DurationInMonths' and 'CreditAmount', the distributions for each target class show a positive skewness (the mean is higher than the mode). In the target class "bad payers" however, the skewness seems to be higher (mean is further away from the mode). Besides, in the joint scatter plot DurationInMonths-CreditAmount, the concentration of "bad payers" is also higher for higher values of 
CreditAmount, specially for higher CreditAmount and lower DurationInMonths values.

#### 3. Other Numerical Features
It is possible to note that all numerical feature except for "Age", "DurationInMonths" and "CreditAmount" have only few discrete possible values (ordinal values). Therefore, a "Spearman" correlation measure is more suitable for correlation analysis, as shown below.

In [ ]:
numerical_features = dataset_df.select_dtypes(include=["int64", "float64"])
numerical_features["CreditRisk"] = dataset_df["CreditRiskClass"].replace(to_replace={"Good": 0, "Bad": 1})

plt.figure(figsize=(7, 6))
sns.heatmap(data=numerical_features.corr(method='spearman'), annot=True)

In the heatmap above, it is possible to see that the DurationInMonths and CreditAmount are the features with the highest correlation in absolute value, followed by "InstallmentRate-CreditAmount", "ResidenceSince-Age" and "PeopleLiableMaintenance-Age".

What regards ranked correlation with the credit risk, with "0" denoting low risk (good payer) and "1" denoting high risk (bad payer), we can see that no feature has a high correlation in absolute value. The highest ones are "DurationInMonths", "Age", "Credit Amount", "InstallmentRate". The highest one (DurationInMonths) has only "-0.27". This suggests that techniques such as Logistic Regression would not perform good. As seen below, it would be harder to fit a logistic function in the graphics below. 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12,4))

sns.scatterplot(ax=axes[0], x="CreditAmount", y="CreditRiskClass", data=dataset_df, hue="CreditRiskClass")
axes[0].grid()

sns.scatterplot(ax=axes[1], x="DurationInMonths", y="CreditRiskClass", data=dataset_df, hue="CreditRiskClass")
axes[1].grid()
axes[1].set_ylabel("")

sns.scatterplot(ax=axes[2], x="Age", y="CreditRiskClass", data=dataset_df, hue="CreditRiskClass")
axes[2].grid()
axes[2].set_ylabel("")

In [ ]:
sns.countplot(ax=axes[1], x="Gender", hue="CreditRiskClass", data=dataset_df, stat="percent")

### Analysis of Categorical Features

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,4))

sns.countplot(ax = axes[0], x="OtherInstallmentPlans", hue="CreditRiskClass", data=dataset_df)
sns.countplot(ax = axes[1], x="OtherDebtorsGuarantors", hue="CreditRiskClass", data=dataset_df)
sns.countplot(ax = axes[2], x="Housing", hue="CreditRiskClass", data=dataset_df)

max_y = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1], axes[2].get_ylim()[1])

# Set the same y-axis limit for both plots
axes[0].set_ylim(0, max_y)
axes[1].set_ylim(0, max_y)
axes[2].set_ylim(0, max_y)

#create_frequency_table_with_credit_risk(index="OtherInstallmentPlans", df=dataset_df)
create_frequency_table_with_credit_risk(index="OtherDebtorsGuarantors", df=dataset_df)
# create_frequency_table_with_credit_risk(index="Housing", df=dataset_df)

In [ ]:
plt.figure(figsize=(12, 4))
sns.countplot(x="Job", hue="CreditRiskClass", data=dataset_df)
plt.title("Job Qualification")
plt.grid()
plt.show()

create_frequency_table_with_credit_risk(index="Job", df=dataset_df)

It is possible to note that the proportion of "Good" payers and "Bad" payers remains almost the same among different categories of jobs. It is also equal to the general percentage of good and bad payers (i.e. 70%/30%). Therefore, the feature regarding the job qualification/status seems to be not relevant for the classification task.

In [ ]:
create_frequency_table_with_credit_risk(index="CreditHistory", df=dataset_df)

For "critical account history / other credits existing", the rate between the target classes is quite different from the 70%/30% rate. However, for all other categories, the rate doesn't seem to change too much. Such result suggests that the relevance of this feature should be analyzed more carefully.

In [ ]:
plt.figure(figsize=(8,4))

order_checking = ['Less than 0 DM', '0 to 200 DM', '200 DM or more', 'No checking account']

sns.countplot(x="CheckingAccountStatus", hue="CreditRiskClass", data=dataset_df, order=order_checking)

create_frequency_table_with_credit_risk(index="CheckingAccountStatus", df=dataset_df)

The feature "CheckingAccountStatus" has apparently a good capability to distinguish between the two classes. 

1. The proportion of "bad payers" in "No checking account" is very small. It is only 11%, what is quite different from the general 30/70% rate
2. The rate of the two target classes is almost the same in the category "Less than 0 DM". This suggests that the there is significant increase of chance of a person being a bad payer if it has a negative checking account.
3. The information of positive checking account offers a slight increase on the risk probability

Such result suggests that this feature is relevant for the classification task.

In [ ]:
plt.figure(figsize=(12,4))
order_savings = ['Less than 100 DM','100 to 500 DM','500 DM or more', 'Unknown / no savings account']
sns.countplot(x="SavingsAccountBonds", hue="CreditRiskClass", data=dataset_df, order=order_savings)

create_frequency_table_with_credit_risk(index="SavingsAccountBonds", df=dataset_df)

## Hypothesis Test

## Classification Model for Prediction

#### Pre-processing

#### Data-Splitting

#### Model Selection and Fitting

#### Performance Analysis

## Conclusion

## Citation